<div style="text-align: center;"><h1> 🔊 Audio MNIST Tutorial - PyTorch 🔊 </h1></div>

<div style="text-align: center; margin-top: 50px"> <img src="./images/hello-again.gif" width="300"> </div>

        LA SUITE

Maintenant que le traitement des données a été effectué, nous allons créer, entraîner et tester notre futur modèle. J'espère que tu as révisé ton cours sur les CNNs car c'est le même type de modèle que nous allons créer.

        LES BIBLIOTHÈQUES

Comme je l'avais précisé dans l'intro de la première partie, nous allons utiliser PyTorch pour créer notre réseau de neurones artificiels. De plus, la bibliothèque va aussi nous permettre de créer la partie convolution de notre réseaux.

On va aussi réutiliser "matplotlib" en plus de "sklearn" et "intertools" pour tracer et visualiser des données sous forme de graphiques afin de mieux comprendre comment fonctionne notre modèle.

Enfin, on utilise aussi "os" et "numpy" pour charger les données traitées précédemment et les utiliser dans le modèle.

In [ ]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset

import os
import numpy as np
from numpy import loadtxt

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

        RÉCUPÉRATION DES DONNÉES
        
Tout d'abord, nous allons charger les données traitées dans des listes afin de pouvoir les utiliser plus tard.

<div style="text-align: center; margin-top: 50px"> <img src="./images/i-have-the-data.gif" width="500"> </div>

In [ ]:
parent_dir = './data'

train_dir = parent_dir + '/train'
test_dir = parent_dir + '/test'

list_train_files_names = []
list_test_files_names = []

# Récupération des chemins vers tout les fichiers
for i in range(10):
    path = train_dir + '/' + str(i)
    list_train_files_names.append(os.listdir(path))

for i in range(10):
    path = test_dir + '/' + str(i)
    list_test_files_names.append(os.listdir(path))

feature_train = []
label_train = []
feature_test = []
label_test = []

# Chargement des données dans des tableaux de matrices
for i in range(len(list_train_files_names)):
    for j in range(len(list_train_files_names[i])):
        data = loadtxt(train_dir + '/' + str(i) + '/' + list_train_files_names[i][j], delimiter=',')
        feature_train.append(data)
        label_train.append(i)

for i in range(len(list_test_files_names)):
    for j in range(len(list_test_files_names[i])):
        data = loadtxt(test_dir + '/' + str(i) + '/' + list_test_files_names[i][j], delimiter=',')
        feature_test.append(data)
        label_test.append(i)

        DERNIÈRE MODIFICATION

Comme le titre l'indique, nous n'avons pas totalement fini de modifier les données avant de les utilisées. En effet, il reste trois derniers détails avant que nos données soit parfaites.

Tout d'abord, il faut normaliser les données pour ainsi accélérer l'entraînement du modèle en réduisant le décalage de covariable interne. Ensuite, nous allons ajouter une dimension aux tenseurs qui va servir d'emplacement pour les futurs "features maps" générés par le CNN. Enfin, on modifie le type des données afin d'être compatible avec celles utilisées par PyTorch.

In [ ]:
# Normalisation des données

In [ ]:
# Ajout d'une dimension aux tenseurs
feature_train = np.expand_dims(feature_train, 1)
label_train = np.array(label_train)

feature_test = np.expand_dims(feature_test, 1)
label_test = np.array(label_test)

In [ ]:
print(len(feature_train))
print(len(feature_test))

print(type(feature_train[0][0][0]))
print(type(label_train[0]))

In [ ]:
print(type(feature_train[0][0][0][0]))
print(type(label_train[0]))

In [ ]:
# Modification des types de variables utilisées
feature_train = np.float32(feature_train)
feature_test = np.float32(feature_test)
label_train = np.int64(label_train)
label_test = np.int64(label_test)

print(type(feature_train[0][0][0][0]))
print(type(label_train[0]))

        CRÉATION DES DATASETS

Le code pour le traitement des échantillons de données peut devenir désordonné et difficile à maintenir. Nous voulons idéalement que notre code de dataset soit découplé de notre code d'entraînement de modèle pour une meilleure lisibilité et modularité. Heuresement, PyTorch fournit deux primitives de données : "torch.utils.data.DataLoader" et "torch.utils.data.Dataset" qui vous permettent d'utiliser des ensembles de données préchargés ainsi que vos propres données. Dataset stocke les échantillons et leurs étiquettes correspondantes, et DataLoader enroule un itérable autour du Dataset pour permettre un accès facile aux échantillons.

Pour ce tutoriel, nous allons créer des Datasets personnalisées. Les classes créés doivent hériter de la classe Dataset et surcharger 3 fonctions:  __init__, __len__ et __getitem__.

La fonction __init__ est exécutée une fois lors de l'instanciation de l'objet Dataset. Souvent, elle permet d'initialisée le répertoire contenant les données, le répertoire des étiquettes si l'apprentissage est supervisé, et des méthodes de transformations des données.

La fonction __len__ renvoie le nombre d'échantillons dans notre jeu de données.

La fonction __getitem__ charge et renvoie un échantillon de l'ensemble de données à l'indice idx donné.

<div style="text-align: center; margin-top: 50px"> <img src="./images/my-data.gif" width="500"> </div>

In [ ]:
# Création d'une classe AuioMNISTDataset
class AudioMNISTDataset(Dataset):
    
    def __init__(self, features, labels, transform=None, target_transform=None):
        self.features = features
        self.labels = labels
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]
        feature = torch.from_numpy(feature)
        if self.transform:
            feature = self.transform(feature)
        if self.target_transform:
            label = self.target_transform(label)
        return feature, label

In [ ]:
# Création de deux Datasets : un pour l'entraînement du modèle et un pour le test
train_dataset = AudioMNISTDataset(feature_train, label_train, transform=nn.BatchNorm1d(64))
test_dataset = AudioMNISTDataset(feature_test, label_test, transform=nn.BatchNorm1d(64))

print(train_dataset[0])
print(test_dataset[0])

In [ ]:
# Création des DataLoader qui utilisent les deux Datasets créés
# Chaque itération sur les DataLoaders renvoie un lot de train_features et train_labels (contenant respectivement batch_size=64 features et labels).
# Parce que nous avons spécifié shuffle=True, les données sont mélangées
batch_size = 64;
loaders = {
    'train' : torch.utils.data.DataLoader(train_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=True),
    
    'test'  : torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=True),
}

        CRÉATION DU MODÈLE

Maintenant que les données sont prêtes à être utilisées, nous allons créer le réseau de neurones au cœur de ce tutoriel. Avec PyTorch, les réseaux de neurones sont composés de couches/modules qui effectuent des opérations sur les données. L'espace de nom torch.nn fournit tous les éléments de base dont vous avez besoin pour construire votre propre réseau neuronal. Chaque module dans PyTorch sous-classe le module nn.Module. Un réseau neuronal est un module en soi qui se compose d'autres modules (couches). Cette structure imbriquée permet de construire et de gérer facilement des architectures complexes.

Pour ce tutoriel, nous allons créer un CNN (Convolutional Neural Network) pour traîter nos données. /* explication de l'utilisation d'un CNN */

<div style="text-align: center; margin-top: 50px"> <img src="./images/confused-travolta.gif" width="500"> </div>

In [ ]:
# Création de la classe contenant les couches du réseau de neurones
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=32,            
                kernel_size=3,              
                stride=1,                   
                padding=1,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(
                in_channels=32,              
                out_channels=32,            
                kernel_size=3,              
                stride=1,                   
                padding=1,                  
            ),   
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        # fully connected layer, output 10 classes
        self.out = nn.Linear(32 * 16 * 16, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # flatten the output of conv2 to (batch_size, 32 * 16 * 16)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output

In [ ]:
# Création du modèle, de la fonction loss et de l'optimizer qui vont être utilisés pour entraîner le modèle
cnn = CNN()
loss_func = nn.CrossEntropyLoss()   
optimizer = optim.Adam(cnn.parameters(), lr = 0.01) 

        L'ENTRAINEMENT DU MODÈLE

Maintenant que la structure du modèle est défini, on va passer à son entraînement (j'espère que vous avez un GPU sur vous ^^). Pour entraîner le réseau, nous allons tout simplement créer une fonction train() qui va faire passer un batch du Dataset d'entrainement dans le réseau et corriger les paramètres du modèle grâce à l'optimizer et la loss définis plutôt.

<div style="text-align: center; margin-top: 50px"> <img src="./images/train.gif" width="500"> </div>

In [ ]:
# Une époque correspond à un passage complet du Dataset dans le réseau.
num_epochs = 3

# Fonction pour entraîner le modèle
def train(num_epochs, cnn, loaders):
    
    # Indique à votre modèle que vous êtes en train de le former. Ainsi, les couches comme dropout, batchnorm etc. qui se comportent différemment dans les procédures d'entraînement et de test savent ce qui se passe et peuvent donc se comporter en conséquence.
    cnn.train()
    
    total_step = len(loaders['train'])

    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            
            # Récupération de la prédiction d'un batch de donnée
            output = cnn(images) 
            # Calcul de la fonction coût de la prédiction            
            loss = loss_func(output, labels)
            # Réinitialise les gradients 
            optimizer.zero_grad()           
            # Backpropagation et calcul des gradients 
            loss.backward()    
            # Application des gradients pour corriger les paramètres du modèle          
            optimizer.step() 

            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, total_step, total_step, loss.item()))

train(num_epochs, cnn, loaders)

print('Model has been trained {} times with {} datas'.format(num_epochs, len(train_dataset)))

        LE TEST DU MODÈLE

Et voilà, nous venons de créer et d'entraîner un réseau de neurones. Mais maintenant, nous devons tester son efficacité. Dans notre cas, le modèle résoud un problème de classification. Nous pouvons donc simplement calculer le taux de réussite de prédiction sur le Dataset de test afin de nous faire une idée sur son efficacité.

Bien sûr, nous pouvons faire d'autres tests afin de vérifier d'autres aspects du modèle. Par exemple créer une matrice de confusion à partir des prédictions des données d'entraînements. Cette matrice peut se révéler très pratique pour connaître les faiblesses de notre modèle, comme la classe que le modèle à le plus de mal à prédire.

<div style="text-align: center; margin-top: 50px"> <img src="./images/test.gif" width="500"> </div>

In [ ]:
# Fonction pour tester le modèle
def test():
    all_preds = torch.tensor([])
    all_labels = torch.tensor([])
    cnn.eval()
    accuracy = 0
    with torch.no_grad():
        for images, labels in loaders['test']:
            test_output = cnn(images)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            accuracy += (pred_y == labels).sum().item() / float(labels.size(0))

            all_preds = torch.cat((all_preds, pred_y),dim=0)
            all_labels = torch.cat((all_labels, labels),dim=0)

    accuracy /= len(loaders['test'])
    print('Test Accuracy of the model on the {} test : {:.2f}%'.format(len(test_dataset), (accuracy*100)))
    return all_labels, all_preds

all_labels, all_preds = test()

In [ ]:
# Fonction pour afficher la matrice de confucion des données test
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    
all_labels = all_labels.type(torch.int64)
all_preds = all_preds.type(torch.int64)
confusion_mat = confusion_matrix(all_labels, all_preds)
labels = np.array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
plt.figure(figsize=(10,10))
plot_confusion_matrix(confusion_mat, labels)

        SAUVEGARDE DU MODÈLE

Enfin, maintenant que nous avons vérifié que le modèle est très performant, nous pouvons le sauvegarder afin de l'utiliser plus tard.

<div style="text-align: center; margin-top: 50px"> <img src="./images/victory.gif" width="500"> </div>

In [ ]:
# Sauvegarde du modèle
torch.save(cnn.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

In [ ]:
# Chargement du modèle
saved_cnn = CNN()
saved_cnn.load_state_dict(torch.load("model.pth"))